In [13]:
import math
import numpy as np
import scipy.linalg
from scipy.linalg import pinv, inv


def l1(x: np.array, y: np.array):
    if x.shape != y.shape:
        raise Exception('Shapes dont match')

    s = 0

    for i in range(len(x)):
        s += abs(y[i] - x[i])
    return s


def l2(x: np.array, y: np.array):
    if x.shape != y.shape:
        raise Exception('Shapes dont match')

    s = 0

    for i in range(len(x)):
        s += (y[i] - x[i]) ** 2
    return s ** (1 / 2)


def linf(x: np.array, y: np.array):
    if x.shape != y.shape:
        raise Exception('Shapes dont match')

    k = x.shape[0]
    s = 0

    for i in range(k):
        s += abs(y[i] - x[i]) ** k
    return s ** (1 / k)


def mean_vector(x):
    """
    na wejsciu podwany jest stojacy wektor np [[1], [1]] a nie [1, 1]
    :param x:
    :return:
    """
    n = x.shape[0]
    mean = np.zeros((1, x.shape[1]))

    for i in range(n):
        mean += x[i]

    mean /= n
    return mean


def covariation_matrix(x: np.array):
    """
    przyjmuje punkt x jako stojacy np. [[1], [2]] a nie [[1, 2]]
    :param x:
    :return:
    """
    n = x.shape[0]
    print(n)

    mean = mean_vector(x)

    print("srednia mi")
    print(mean)

    summer = np.zeros((x.shape[1], x.shape[1]))

    for i in range(n):
        xx = x[i] - mean
        summer += xx.T @ xx

    summer /= n

    print('macierz kowariancji:')
    print(summer)

    for i in range(summer.shape[0]):
        for j in range(summer.shape[1]):
            if i != j:
                print("pearson dla ", i + 1, j + 1)
                print(round(summer[i][j] / ((summer[i][i] ** (1 / 2)) * (summer[j][j] ** (1 / 2))), 3))

    return summer



def mahalanabis_distance(x: np.array, c: np.array, mi: np.array):
    """
    przyjmuje punkt x jako lezacy np. [1, 2] a nie [[1], [2]]
    """
    cinv = inv(c)
    xx = x - mi

    return xx @ cinv @ xx.T


In [3]:
a = np.array([[0, 0]]).T
b = np.array([[1, 1]]).T
c = np.array([[0, 0.7]]).T

print(l1(a, c))
print(l1(b, c))

print(l2(a, c))
print(l2(b, c))

print(linf(a, c))
print(linf(b, c))

[0.7]
[1.3]
[0.7]
[1.04403065]
[0.7]
[1.04403065]


In [4]:
a = np.array([[-2, -2], [-1, 1], [1, -1], [2, 2]])
c = covariation_matrix(a)
c

4
srednia mi
[[0. 0.]]
macierz kowariancji:
[[2.5 1.5]
 [1.5 2.5]]
pearson dla  1 2
0.6
pearson dla  2 1
0.6


array([[2.5, 1.5],
       [1.5, 2.5]])

In [6]:
miX = mean_vector(a)

dd = np.array([4, -4])

ddd = np.array([4, 4])

mahalanabis_distance(dd, c, miX)

array([[32.]])

In [7]:
mahalanabis_distance(ddd, c, miX)

array([[8.]])

In [16]:
prupki = np.array([[2, 4, 6, 4], [1, 0, 1, 2]])
print(prupki)

miP = mean_vector(prupki.T)

maciesz = covariation_matrix(prupki.T)

[[2 4 6 4]
 [1 0 1 2]]
4
srednia mi
[[4. 1.]]
macierz kowariancji:
[[2.  0. ]
 [0.  0.5]]
pearson dla  1 2
0.0
pearson dla  2 1
0.0


In [18]:
inv_maciesz = pinv(maciesz)

array([[0.5, 0. ],
       [0. , 2. ]])

In [27]:
punkt = np.array([[4, 4]])
mahalanabis_distance(punkt, maciesz, miP)

# l2(punkt.T, miP.T)

array([[18.]])